# NEOM Managed Aquifer Recharge (Injection Wells)

*James McCreight, Amgad Ellaboudy, Josh Daniel*

We focus on suitability mapping for locations of injection wells using large-scale datasets. It seems that injection wells are a preferable strategy over check dams in the NEOM region because they would reduce large evaporative losses on the surface and in the unconfined aquifer/soil. Though, there could be certain situations where a check dam might make more sense. The check dam approach would extend the injection well suitability mapping approach here, because we a re not incorporating geological/aquifer datasets. For check dams soil infiltration and water retention scores would additional layers for suitability mapping. 

Injection Wells Layers/Predictors:
1. Constraint: Slope in [1,9] degrees
1. Constraint: Potential runoff contributing areas smalle than typical convective scales (<= 460km2)
1. Constraint: Potential runoff in [.02, 2] cms
1. Suitability: Potential runoff: more is better.

1. HAND?

Additional Predictors for Check Dam suitability
1. Check dam suitability: Upper soil type is favorable for water transmission
1. Check dam suitability: Lower soil type is favorable for water retention

## Preliminaries

In [1]:
import ee
import numpy as np
from statistics import mean
from earthshot import water_viz as vis
from earthshot import water_common as common
from earthshot import normalize as norm

from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show
from bokeh.models import Range1d

Loading BokehJS ...

In [2]:
#ee.Authenticate()
ee.Initialize()

In [3]:
# Some common parameters
month_names = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
days_per_month = [
    31, 28, 31, 30, 31, 30,
    31, 31, 30, 31, 30, 31] 
seconds_per_month = [ dd* 24*60*60 for dd in days_per_month ]

bbox_name = 'neom'
bbox = common.bboxes()[bbox_name]

## Predictors
**Terrain slope** derived from [MERIT Hydro](https://developers.google.com/earth-engine/datasets/catalog/MERIT_Hydro_v1_0_1#description)

In [4]:
slope_min = 1
slope_max = 9
slope_deg = ee.Image('users/jamesmcc/merit_slope/merit_terrain_slope')
slope_mask = slope_deg.lte(slope_max).And(slope_deg.gte(slope_min))
slope_inv = slope_deg.updateMask(slope_mask).pow(-1) 
slope_inv_scaled = norm.img_scale(slope_inv, area_of_interest=bbox) 
norm.img_range(slope_inv_scaled, area_of_interest=bbox)

[0, 0.9999999999999999]

**Contributing area** from MERIT Hydro. Potential runoff calculation assumes that rainfall is uniform across the contributing area. If rainfall in NEOM region is dominated by convective/thunder storms, then potential runoff is only valid for contributing areas of this scale. Use a standard radius of such storms (7.5miles = 12km) as the maximum contributing area (177mi2 = 460km2) for which potential runoff is valid.

In [5]:
max_contrib_area_km2 = 460
merit_drain_area_km2 = ee.Image("MERIT/Hydro/v1_0_1").select('upa')
drain_area_mask = merit_drain_area_km2.lte(max_contrib_area_km2)

**Potential runoff** monthly cilmatologies [from ERA5 and Merit Hydro](https://code.earthengine.google.com/?asset=users/jamesmcc/era5_merit_potential_sfc_runoff_neom_climatology)

In [6]:
pot_runoff_clim = ee.ImageCollection(
    'users/jamesmcc/era5_merit_potential_sfc_runoff_neom_climatology')

Inspect the monthly potential runoff to understand how to reduce it.

In [7]:
pot_runoff_min_cms = .001 ## have to set this low to get histograms in some months
pot_runoff_max_cms = 2
plots = []
for month0 in range(12):
    month = month0 + 1
    month_name = month_names[month - 1]

    pot_runoff_cms = (
        pot_runoff_clim
        .filter(ee.Filter.eq('month', month))
        .filter(ee.Filter.eq('variable', 'potential_sfc_runoff_mon_clim_cms'))
        .first())
    
    # Apply all the constraints
    pot_runoff_cms_mask = (
        pot_runoff_cms.gte(pot_runoff_min_cms).And(
            pot_runoff_cms.lte(pot_runoff_max_cms))
        .updateMask(drain_area_mask)
        .updateMask(slope_mask))

    pot_runoff_hist = ee.Dictionary(
        pot_runoff_cms
        .updateMask(pot_runoff_cms_mask)
        .reduceRegion(
            ee.Reducer.histogram()).get('potential_sfc_runoff_mon_clim_cms'))
    hist = np.array(pot_runoff_hist.get('histogram').getInfo())
    edges = (
        (np.array(list(range(len(hist) + 1))) * 
         np.array(pot_runoff_hist.get('bucketWidth').getInfo())) + 
        np.array(pot_runoff_hist.get('bucketMin').getInfo()) )

    p = figure(
        title='NEOM Potential Runoff (cms): ' + month_name, tools="", 
        y_axis_type="log", 
        # x_axis_type="log",
        y_range=Range1d(1, 45000),
        x_range=Range1d(pot_runoff_min_cms, pot_runoff_max_cms),
        background_fill_color="white")
    _ = p.quad(
        top=hist,
        bottom=1,
        left=edges[:-1],
        right=edges[1:],
        fill_color="navy",
        line_color="navy",
        alpha=1.)
    plots = plots + [p]

In [8]:
show(gridplot([plots[9:12], plots[0:3], plots[3:6], plots[6:9]], plot_width=250, plot_height=250))

Reduce the annual cycle by taking the maximum of the potential runoff over the months (image collection).

In [9]:
pot_runoff_min_cms = .1 ## setting this to the range we actually want
pot_runoff_max_cms = 2

pot_runoff_clim_max_cms = ee.Image(pot_runoff_clim.max())

# Apply all the constraints
pot_runoff_clim_max_cms_mask = (
    pot_runoff_clim_max_cms.gte(pot_runoff_min_cms).And(
        pot_runoff_cms.lte(pot_runoff_max_cms))
    .updateMask(drain_area_mask)
    .updateMask(slope_mask))

pot_runoff_final_cms = pot_runoff_clim_max_cms.updateMask(pot_runoff_clim_max_cms_mask)

pot_runoff_hist = ee.Dictionary(
    pot_runoff_final_cms
    .reduceRegion(
        ee.Reducer.histogram(), scale=90).get('potential_sfc_runoff_mon_clim_cms'))

hist = np.array(pot_runoff_hist.get('histogram').getInfo())
edges = (
    (np.array(list(range(len(hist) + 1))) * 
     np.array(pot_runoff_hist.get('bucketWidth').getInfo())) + 
    np.array(pot_runoff_hist.get('bucketMin').getInfo()) )

p = figure(
    title='NEOM Potential Runoff Climatological Max (cms): ', 
    y_axis_type="log", 
    # x_axis_type="log",
    y_range=Range1d(1, 300),
    x_range=Range1d(pot_runoff_min_cms, pot_runoff_max_cms),
    background_fill_color="white")
_ = p.quad(
    top=hist,
    bottom=1,
    left=edges[:-1],
    right=edges[1:],
    fill_color="navy",
    line_color="navy",
    alpha=1.)

In [10]:
show(p)

In [11]:
pot_runoff_range = norm.img_col_range(ee.ImageCollection(pot_runoff_final_cms), area_of_interest=bbox)
pot_runoff_scaled = pot_runoff_final_cms.subtract(pot_runoff_range[0]).divide(pot_runoff_range[1]-pot_runoff_range[0])
vis_range_scale = norm.img_range(pot_runoff_scaled, area_of_interest=bbox)

In [12]:
box_corners = common.bboxes()['neom'].toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])

In [13]:
vis_range = vis_range_scale
vis_image = pot_runoff_scaled
palette_name = 'Dark2'
palette_len = 4
palette = vis.brewer[palette_name][palette_len][::]
vis.legend(
    palette=palette, minimum=vis_range[0], maximum=vis_range[1], title='NEOM Potenital Runoff Score')

vis_params = {
    'min': vis_range[0], 'max': vis_range[1], 
    'dimensions': 512,
    'palette': palette}

In [14]:
the_map = vis.folium_map(location=[center_lat, center_lon], zoom_start=8, height=700)
the_map.add_ee_layer(pot_runoff_final_cms, vis_params, 'NEOM Potential Runoff Score')
vis.folium_display(the_map)

In [15]:
# display(vis.Image(url = vis_image.getThumbURL(vis_params)))

## Calculate the MAR Score

In [16]:
score = (
    slope_inv_scaled.multiply(.333)
    .add(pot_runoff_scaled.multiply(.666)))
score = norm.img_scale(score, area_of_interest=bbox).rename('score')
norm.img_range(score, area_of_interest=bbox)

[0, 1]

In [17]:
# Plot a histogram of the scores
score_hist = ee.Dictionary(
    score.clip(bbox)
    .reduceRegion(
        ee.Reducer.histogram()).get('score'))

hist = np.array(score_hist.get('histogram').getInfo())
edges = (
    (np.array(list(range(len(hist) + 1))) * 
     np.array(score_hist.get('bucketWidth').getInfo())) + 
    np.array(score_hist.get('bucketMin').getInfo()) )

p = figure(
    title='NEOM MAR Score', 
    background_fill_color="white", height = 300)
_ = p.quad(
    top=hist,
    bottom=0,
    left=edges[:-1],
    right=edges[1:],
    fill_color="navy",
    line_color="navy",
    alpha=1.)
show(p)

In [18]:
vis_range = [0,1]
vis_image = score
palette_name = 'Dark2'
palette_len = 4
palette = vis.brewer[palette_name][palette_len][::]
vis.legend(
    palette=palette, minimum=vis_range[0], maximum=vis_range[1], title='NEOM MAR Score')

vis_params = {
    'min': vis_range[0], 'max': vis_range[1], 
    'dimensions': 512,
    'palette': palette}

In [19]:
the_map = vis.folium_map(location=[center_lat, center_lon], zoom_start=8, height=700)
the_map.add_ee_layer(score, vis_params, 'NEOM MAR Score')
vis.folium_display(the_map)

## Export map

In [20]:
collection_name = 'users/jamesmcc/NEOM'
max_pixels = 100000000000  #For global is quite large
description = "NEOM MAR Score"

In [21]:
#_ = ee.batch.Export.image.toAsset(
#        score,
#        description=description, 
#        assetId=collection_name + '/' + 'NEOM_MAR_Score',
#        region=bbox, 
#        scale=100,
#        maxPixels=max_pixels).start()

## Notes

We can not calculate/sum the potential runoff score as it's highly spatially correlated. Need a proper hydrologic processing system to do that. 

The potential runoff and the score also reflect the ad hoc nature of the potential runoff calculation (again, a proper hydrologic system / model would fix this). Potential runoff should be strictly increasing in the downstream direction along the stream channel network but does not appear this way as runoff can vary abruptly with ERA5 model grid cell. The ad hoc calculation of potential runoff assumes that the runoff at a point is homogeneous/representative of its upstream area. This is not a good assumption in dry regions or seasons with convective precipitation and is why the total contributing area constraint is applied. 